In [1]:
## 20220550 윤명섭
!pip install folium

In [20]:
import folium

latitude = 37.566535
longitude = 126.977941

map = folium.Map(location=[latitude, longitude], zoom_start=10)

map.save('./seoul_map.html')

In [15]:
## 20220550 윤명섭
import pandas as pd
data = pd.read_csv("/content/seoul-metro-2021.logs.csv")
print(data.head(), "\n")
print(data.info())


                       timestamp  station_code  people_in  people_out
0  2021-01-01T05:00:00.000+09:00           150         86          85
1  2021-01-01T06:00:00.000+09:00           150        111         355
2  2021-01-01T07:00:00.000+09:00           150        157         438
3  2021-01-01T08:00:00.000+09:00           150        306         592
4  2021-01-01T09:00:00.000+09:00           150        333         841 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941553 entries, 0 to 1941552
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   timestamp     object
 1   station_code  int64 
 2   people_in     int64 
 3   people_out    int64 
dtypes: int64(3), object(1)
memory usage: 59.3+ MB
None


In [16]:
# station code 기준으로, people_in(승차 인원), people_out(하차 인원) 집계
GroupedData = data.groupby('station_code')[["people_in", "people_out"]].sum()
# 결과 출력
print(GroupedData.head())

              people_in  people_out
station_code                       
150            12697273    12109991
151             5997344     6030491
152             9638952     9284693
153             7704599     7090896
154             6651283     6609055


In [17]:
StationInfo = pd.read_csv("/content/seoul-metro-station-info.csv")
print(StationInfo.head(), "\n")
print(StationInfo.info())

   station.code station.fr_code  line.num line.name line.name_sub  \
0           158             124         1       1호선        지하철1호선   
1           157             125         1       1호선        지하철1호선   
2           156             126         1       1호선        지하철1호선   
3           159             127         1       1호선        지하철1호선   
4           155             128         1       1호선        지하철1호선   

   line.station_seq station.name_full station.name station.name_chc  \
0                 1      청량리(서울시립대입구)  청량리|서울시립대입구              祭基洞   
1                 2               제기동          제기동              新設洞   
2                 3               신설동          신설동              東廟앞   
3                 4               동묘앞          동묘앞     淸凉里(서울市立大入口)   
4                 5               동대문          동대문              東大門   

  station.name_chn                     station.name_en station.name_jp  \
0              祭基洞                            Jegidong           チェギドン   
1         

In [18]:
# 필요한 행만 추리기
StationInfo = StationInfo[["station.code", "geo.latitude", "geo.longitude"]]
print(StationInfo.head(), "\n")

# index 변경하기
StationInfo = StationInfo.set_index("station.code")
print(StationInfo.head())

   station.code  geo.latitude  geo.longitude
0           158     37.580178     127.046835
1           157     37.578103     127.034893
2           156     37.575297     127.025087
3           159     37.572627     127.016429
4           155     37.571420     127.009745 

              geo.latitude  geo.longitude
station.code                             
158              37.580178     127.046835
157              37.578103     127.034893
156              37.575297     127.025087
159              37.572627     127.016429
155              37.571420     127.009745


In [19]:
# data set 합치기
JoinedData = GroupedData.join(StationInfo)
print(JoinedData.head())

              people_in  people_out  geo.latitude  geo.longitude
station_code                                                    
150            12697273    12109991     37.554648     126.972559
151             5997344     6030491     37.564718     126.977108
152             9638952     9284693     37.570161     126.982923
153             7704599     7090896     37.571607     126.991806
154             6651283     6609055     37.570926     127.001849


In [22]:
# 승차용 서울 지도 생성
SeoulIn = folium.Map(location = [37.55, 126.98], zoom_start = 12)

# 지도 확인
print(SeoulIn)

In [23]:
# HitMap plugins 모듈 탑재
from folium.plugins import HeatMap

# HitMap plugins 지도에 추가하기
HeatMap(data = JoinedData[["geo.latitude", "geo.longitude", "people_in"]]).add_to(SeoulIn)
# 지도 확인
print(SeoulIn)

In [24]:
# 하차용 서울 지도 생성
SeoulOut = folium.Map(location = [37.55, 126.98], zoom_start = 12)

# HitMap plugins 지도에 추가하기
HeatMap(data = JoinedData[["geo.latitude", "geo.longitude", "people_out"]]).add_to(SeoulOut)
# 지도 확인
print(SeoulOut)

승차  인원과  하차  인원의  히트맵  그래프가  차이가  없어  보이는  이유
 - 데이터를 가공할 때 시간 정보를 포함하지 않았음
 - 지하철역 코드 중심으로 데이터를 하나로 취합하여 그래프에서 차이를 확인하기 어려움
 - 출근 시간과 퇴근 시간이 고려되지 않았기 때문에 확연히 유동인구가 많은 장소를 결정 할 수 없음
 - 결론 - 시간 데이터를 고려해 다시 한 번 데이터를 가공하고 시각화를 다시 진행

In [28]:
# 출근용 데이터 추출
MorningData = data[pd.to_datetime(data.timestamp, errors = "coerce").dt.hour < 9]
print(MorningData.head(), "\n")

# 출근용 데이터 추출
EveningData = data[pd.to_datetime(data.timestamp, errors = "coerce").dt.hour > 17]
print(EveningData.head())

                        timestamp  station_code  people_in  people_out
0   2021-01-01T05:00:00.000+09:00           150         86          85
1   2021-01-01T06:00:00.000+09:00           150        111         355
2   2021-01-01T07:00:00.000+09:00           150        157         438
3   2021-01-01T08:00:00.000+09:00           150        306         592
19  2021-01-01T05:00:00.000+09:00           151         43          40 

                        timestamp  station_code  people_in  people_out
13  2021-01-01T18:00:00.000+09:00           150        658         437
14  2021-01-01T19:00:00.000+09:00           150        579         425
15  2021-01-01T20:00:00.000+09:00           150        479         354
16  2021-01-01T21:00:00.000+09:00           150        510         307
17  2021-01-01T22:00:00.000+09:00           150        445         134


In [29]:
# 역 기준으로 그룹화하여 데이터 더하기
MorningStationSum = MorningData.groupby("station_code").sum()
EveningStationSum = EveningData.groupby("station_code").sum()

In [30]:
MorningJoinedData = MorningStationSum.join(StationInfo)
EveningJoinedData = EveningStationSum.join(StationInfo)

In [31]:
from re import M
# 출근 시간 승차 인원
MorningSeoulIn = folium.Map(location = [37.55, 126.98], zoom_start = 12)
HeatMap(data = MorningJoinedData[["geo.latitude", "geo.longitude", "people_in"]]).add_to(MorningSeoulIn)
print(MorningSeoulIn)

# 출근 시간 하차 인원
MorningSeoulOut = folium.Map(location = [37.55, 126.98], zoom_start= 12)
HeatMap(data = MorningJoinedData[["geo.latitude", "geo.longitude", "people_out"]]).add_to(MorningSeoulOut)
print(MorningSeoulOut)

# 퇴근 시간 승차 인원
EveningSeoulIn = folium.Map(location = [37.55, 126.98], zoom_start = 12)
HeatMap(data = EveningJoinedData[["geo.latitude", "geo.longitude", "people_in"]]).add_to(EveningSeoulIn)
print(EveningSeoulIn)

# 퇴근 시간 하차 인원
EveningSeoulOut = folium.Map(location = [37.55, 126.98], zoom_start= 12)
HeatMap(data = EveningJoinedData[["geo.latitude", "geo.longitude", "people_out"]]).add_to(EveningSeoulOut)
print(EveningSeoulOut)

서울 지하철 유동인구 현황 분석

출근 시간에 지하철에서 하차하여 직장으로 이동하는 사람이 가장 많은 지하철역

강북의 시청역과 서울역, 강남의 강남역과 디지털구로단지역

--==--

퇴근 시간에 지하철에서 승차하는 사람이 가장 많은 지하철역: 강남역

하차하는 사람이 가장 많은 지하철역: 신림역